Welcome to the coding demonstration from the Plant Cell Atlas. As you can see, you are running a Jupyter Notebook through Google Colab, a free service from Google. A Jupyter Notebook is a useful coding tool that allows you to run code in small blocks, which can be helpful for data science  where you make a lot of small changes and need to see the results midway through. You also have markdown blocks, like this one, which you can use to make headings and other text descriptions. IDEs will even let you jump between headings like bookmarks. 

---

While we are currently on Google Colab, you can also run Jupyter Notebooks locally or on a remote server. 

---

Although Jupyter Notebooks are very useful tools, its important to keep in mind that they have some drawbacks. Code blocks are executed in the order you run them, and can be executed multiple times or in arbitrary orders, so if you aren't careful you can end up making unintentional changes to your code without even realizing it. You should make sure you run cells in the order they are in your notebook to ensure your codes works as you expect it to. 

First, we will look at some heading sizes available. These are done by putting a number sign "#" at the start of the line. More "#" makes the heading smaller

# Markdown Heading 1 

## Markdown Heading 2 

### Markdown Heading 3 

You can also **bold text** put putting double asterisks around it, and *italicize* with single asterisks, and almost any other text or table formatting. You can also easily insert links, do quotes, and insert images. 

---

Double click on this box to see the source text

In [ ]:
# Number signs set off comments in Python. Below we use "!" to run shell commands from within a Jupyter notebook cell.
# We are fetching our data and installing packages we need. 

!pip install scanpy
!pip install leidenalg
!mkdir data
!wget https://zenodo.org/records/18553930/files/arabidopsis_data.h5ad -O data/arabidopsis_data.h5ad

!cd data
!mkdir write

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import random
import itertools
import h5py

First we are going to set up our random seeds for reproducibility. Some processes initialize randomly, so to ensure that 
we can reproduce our results, we set the random state and will use it in our code. Once you have preliminary results,
you should check that your results are robust to different random states. It is rare to find results that are 
sensitive to the random state. 

In [ ]:
random_state = 1789
np.random.seed(random_state)
random.seed(random_state)

Next we are going to set up some thresholds up front. By defining them early, we tell someone reading out notebook about our assumptions so they aren't buried deep in code. It also lets us change them easily, without having to search through the notebook. 

In [ ]:
# Cell Thresholds
min_genes_per_cell = 2000
min_cells_per_gene = 5
max_different_genes_per_cell = 6000
max_pct_counts_mt = 1
# Highly Variable Gene Selection Thresholds
highly_variable_gene_min_mean = 0.0125
highly_variable_gene_max_mean = 3
highly_variable_gene_min_disp = 1
# Clustering Parameters
number_of_neighbors = 10
n_of_pca_components = 40
clustering_resolution = 0.45

In [ ]:
sc.settings.verbosity = (
    3  # Tell scanpy we want it to be chatty today and inform us what's going on
)
sc.set_figure_params(
    facecolor="white", figsize=(5, 5)
)  # Make figures bigger and on a white background

First, lets read in our data

In [ ]:
adata = sc.read_h5ad(
    "data/arabidopsis_data.h5ad",
)
adata.obs = adata.obs.drop(columns=["fastq_1", "fastq_2", "expected_cells"])
adata.var = adata.var.drop(columns=["genome", "gene_versions", "feature_types"])

Next, in case our annotation has overlapping gene symbols, we are going to make the gene symbols unique for any symbols with duplicates 

In [ ]:
adata.var_names_make_unique()  ## This adds a number after duplicate variable names

Now, lets explore the structure of our data a bit

In [ ]:
adata  # General Object view

As you can see, our AnnData Object stats off pretty empty. Its got the data matrix under adata.X, the cell barcodes and batches under .obs, and the gene symbols and gene IDs under .var. Lets explore those additional aspects of the data

In [ ]:
print(type(adata.obs))  # Prints the datatype of the observation section of the AnnData
display(adata.obs)  # Returns the observation section of the AnnData
print(
    adata.obs["sample"].unique()
)  # Prints the unique values in the "sample" column of the observation section of the AnnData

In [ ]:
print(type(adata.var))  ## Prints the Object type of the Variables of the AnnData
adata.var  ## Returns view of the variables of the AnnData

In [ ]:
print(type(adata.X))  ## Prints the object type of the data matrix of the AnnData
adata.X  ## Returns view of the data matrix of the AnnData

As you can see, we have two familiar data types, Pandas, and one that is a bit different, a sparse matrix. Because a single cell experiment has ~ 30,000 variables for each cell sequence, the matrix rapidly becomes huge as you add cells to it. However, you have zero reads for most genes in most cells, so our matrix is very sparse. Storing each individual zero is computationally taxing, so we use a sparse matrix to manage this behind the scenes for us. However, we will need to keep this in mind for cases and actions that require a dense matrix 

Next, we'll look at some initial info about our experiment, and look at how sparse the matrix is. 

In [ ]:
dense_matrix = pd.DataFrame(
    data=adata.X.todense()
)  # convert dense matrix to sparse to view
dense_matrix.iloc[0:30, :20]  ## View the matrix

In [ ]:
dense_matrix.shape  ## View the dimensions of the matrix

In [ ]:
(dense_matrix > 0).sum().sum()

In [ ]:
# Get the length and width of the matrix
dense_matrix_width = dense_matrix.shape[1]
dense_matrix_length = dense_matrix.shape[0]

total_size_of_matrix = (
    dense_matrix_width * dense_matrix_length
)  ## Calculate total number of values in matrix

# check what positions are non zero
non_zero_dense_matrix = dense_matrix > 0

# Sum the number of non zero positions
filled_posistions = (
    non_zero_dense_matrix.sum().sum()
)  ### The number of cells with a non zero value
percent_of_matrix_filled = (
    filled_posistions / total_size_of_matrix
) * 100  ## You remember what a percentage is
print(
    f"Our Matrix only has a value at {round(percent_of_matrix_filled,3)}% of posistions!"
)  # Round number and print it with an fstring

In [ ]:
sc.pl.highest_expr_genes(
    adata, n_top=20
)  # Use builtin scanpy plotting function to visualize genes with the most reads

Lets take 5 minutes, everyone look up what the first gene does. If you can, look up where it tends to be localized (Suba can help with this). What might this indicate about the data?  Also look at the distribution of counts per cell and think about what that means. What is each dot on the graph? Now pick any After five minutes we will ask everyone what their gene does and if they think it makes sense to be so highly expressed. If you'd like, guess what the experimental treatment for this dataset was. 

This plots our most expressed genes, letting us get an idea if our experiment had anything go dramatically wrong. For example, if your most expressed genes were all stress genes, you might be concerned you mishandled cells prior to sequencing.

In a normal workflow, this might be where you peform background detection. Some RNA has a tendency to leak out of the cells/nuclei and into the ambient fluid. This ambient RNA then gets mixed with every cell's profile. This is especially problematic for marker detection, where broad detection of a gene might cause you to rule it out as a marker. For this, I would recommend CellBender or SoupX. CellBender is substantially more computationally intensive than SoupX. 


Before we keep going, we are going to remove cells and genes that we aren't interested in. For genes, we are only interested in genes that are in our dataset in at least a few cells, a gene in a single cell doesn't tell us much. We are also only interested in cells with enough reads to be able to cluster them. We might also remove genes with high mitochondrial signal (eg. the first gene) as this can indicate the cells are dying or low quality. 

In [ ]:
sc.pp.filter_cells(
    adata, min_genes=min_genes_per_cell
)  # Filter cells with less than 1000 genes expressed
sc.pp.filter_genes(
    adata, min_cells=min_cells_per_gene
)  # Filter genes expressed in less than 5 cells

Now, lets explore our cleaned dataset

In [ ]:
adata.obs

As you can see, we've trimmed out genes that weren't expressed in our data! We also trimmed some cells that have low expression. I will note that our cell threshold is quite high, as this data has already been preprocessed somewhat. We've also added some columns to our Pandas describing our cells and our genes. 



In [ ]:
adata.var["mt"] = adata.var_names.str.startswith("AT1G07590")
# annotate the gene we identified as mitochondrial so we can remove it later

In [ ]:
adata.var

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)  ### This calculates several QC metrics all at once, based on what genes we feed in

In [ ]:
adata.obs

A peak at the metrics we calculated
The number of genes expressed in the count matrix - different because n_genes was calculated before filtering
The total counts per cell
The percentage of counts in mitochondrial genes
 now lets visualize them

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)  ## Use built in plotting function to visualize the QC metrics we calculated

Now we want to remove genes with too many counts, as these could be droplets that got too many cells or another technical artifact. We also want to remove cells with too much mitochondrial DNA, as this indicates the cells are undergoing apoptosis. As an alternative to threshold based approaches, you might use a doublet detector like DoubletFinder or DoubletDetection.

In [ ]:
adata = adata[
    adata.obs.n_genes_by_counts < max_different_genes_per_cell, :
]  ## Remove cells with more than 2500 counts (This is dated don't use this threshold on modern data)
adata = adata[
    adata.obs.pct_counts_mt < max_pct_counts_mt, :
]  # Remove cells with more than 5 % Mito reads

We've finished filtering, so we're gonna save our object at this step. It is important to save intermediate objects in case you have an issue with your pipeline and need to repair it later, or if you just want to re-do some analysis without running the entire thing. Changes to packages might happen that break code and you don't want to have to spend days getting it working again!


In [ ]:
adata.write(
    "write/pbmc3k_filtered.h5ad",
    convert_strings_to_categoricals=False,
)  # Making strings categories saves a lot of space!

Next we will normalize the data , and logarithmize it. 

In [ ]:
sc.pp.normalize_total(
    adata, target_sum=1e4
)  ## Scale counts in each cell to sum to 10,000 for comparisons between cells

In [ ]:
sc.pp.log1p(
    adata
)  ## Log+1 transforms the data - this deals with zero counts since log(0) doesn't exist, without impacting rest of data

Now, to identify cell clusters we need to use genes that are different between cells. A gene that is expressed at the same level in every cell isn't useful for distinguishing them. So lets find our highly variable genes. 

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    min_mean=highly_variable_gene_min_mean,
    max_mean=highly_variable_gene_max_mean,
    min_disp=highly_variable_gene_min_disp,
)  # Identify highly variable genes based on mean and dispersion

This adds a lot of info to our .var, lets check it out. 

In [ ]:
adata.var

In [ ]:
sc.pl.highly_variable_genes(
    adata
)  # Use Built in plotting function to visualize highly variable genes

Now that we've found our highly variable genes, lets begin to reduce the dimensionality of our data I'll explain PCA. We are going to skim over the nearest neighbors, as it is very intutitive. However, we will discuss UMAPs and their limitations as 2D projections of data. 

In [ ]:
sc.pp.pca(
    adata, svd_solver="arpack", random_state=random_state
)  ## Identifies nearest neighbors of each cell based on PCA reduction

In [ ]:
# sc.pl.pca_variance_ratio(adata) #Visualize contribution of PCAs to variance - uncomment if curious will not affect data

In [ ]:
sc.pp.neighbors(
    adata,
    n_neighbors=number_of_neighbors,
    n_pcs=n_of_pca_components,
    random_state=random_state,
)  # Finds Distances in multidimensional space between cells

In [ ]:
sc.tl.umap(
    adata, random_state=random_state
)  # Visualizes cells in multidimensional space as 2d embedding

In [ ]:
sc.tl.leiden(
    adata, resolution=clustering_resolution, random_state=random_state
)  # Clusters cells by distance in multidimensional space

In [ ]:
adata.obs

In [ ]:
sc.pl.umap(adata, color=["leiden"])  # Visiualize Clustering

Before we go on, lets take 10 minutes to play with the clustering settings and see how things change. Try changing resolution, n_neighbors, and n_pca one at a time and see how it changes the data. Do it one and a time and then in combination. Also try changing the highly variable gene selection cut offs. 

In [ ]:
adata.write(
    "/write/pbmc3k_clustered.h5ad",
    convert_strings_to_categoricals=False,
)  # We are saving our object again after clustering

Now lets try and identify one of the clusters, using 3 markers for root cortex cells

In [ ]:
sc.pl.umap(
    adata, color=["leiden", "AT1G27030", "AT1G02850", "AT1G05570"]
)  # Visualize clustering and Attempt to Identify Cortex Cells By expression

Now lets take 5 minutes and let everyone plot some genes on their own. You can takes genes from above, where we looked at the most expressed genes, or pick genes that you know or like. Plot a couple of genes on your own!

In [ ]:
sc.pl.umap(
    adata, color=["#####", "#####", "#####"]
)  # shows embedding with cells colored by expression of 3 marker genes

In [ ]:
adata.obsm

In [ ]:
new_cluster_names = {"1": "Cortex"}
adata.obs["leiden"] = adata.obs["leiden"].replace(
    new_cluster_names
)  ## Replace the cluster name

In [ ]:
sc.pl.umap(
    adata,
    color="leiden",
    legend_loc="on data",
    legend_fontsize="x-small",
    title="Identified Cell Clusters",
)

Now that we have clusters, we can do things like find marker genes, ID cell types, and do differential gene expression. 

One key tool to understand is psuedobulking data. Psuedobulking helps handle the sparsity of single-cell data while also being more intuitive to handle statistically. If you want to compare two cell types, you might think to treat each cell as a sample, and do your statistics that way. However, this would provide you with dramatically more confidence in your estimate than is real, as the cells in a cluster are not independent. Instead, we psuedobulk, allowing us to reduce sparsity and reduce the cells to true idependent samples, based on batch. 

First, we're going to create some fake batches and assign cells to them

In [ ]:
adata.obs["Batch"] = np.random.randint(
    low=1, high=4, size=len(adata.obs)
)  # Assign random batches - do you know why high is 4?
adata.obs.head(40)

In [ ]:
sc.pl.umap(
    adata, color="Batch", legend_fontsize="x-small", title="Fake Batches", s=400
)  # Notice the overplotting when using the batch as an integer!

In [ ]:
adata.obs["Batch Str"] = adata.obs["Batch"].astype(
    str
)  # Change the batch type to string

In [ ]:
sc.pl.umap(
    adata, color="Batch Str", legend_fontsize="x-small", title="Fake Batches", s=400
)  # overplotting resolved! Think of this when plotting a metadata variable on single cell data

Now we will loop through the batches and build a dataframe. Note that this code assumes all clusters contain all batches - This might not be true!

In [ ]:
clusters = adata.obs["leiden"].unique()  # Get cluster names
batches = adata.obs["Batch"].unique()  # Get batch IDs
all_samples = list(
    itertools.product(clusters, batches)
)  # Generate list of all combinations of the above

In [ ]:
all_samples

In [ ]:
psuedobulk_df = pd.DataFrame(
    index=adata.var_names
)  ## Make a base dataframe index we will add stuff on to later
psuedobulk_df

In [ ]:
for batch_type in all_samples:

    ## Read in the Names so our code is easy to understand
    current_cluster = batch_type[0]
    current_batch = batch_type[1]

    ## Calculate the Psuedobulked mean
    cells_matching_batch_and_cluster = adata[
        (adata.obs["leiden"] == current_cluster) & (adata.obs["Batch"] == current_batch)
    ]
    mean_of_genes = cells_matching_batch_and_cluster.X.mean(axis=0).tolist()[0]

    name_of_combo = (
        "Cluster number " + current_cluster + " " + "Batch Number " + str(current_batch)
    )
    psuedobulk_df[name_of_combo] = mean_of_genes

In [ ]:
adata.write(
    "write/pbmc3k_final.h5ad",
    convert_strings_to_categoricals=False,
)  # Save our final object!

In [ ]:
psuedobulk_df.head(10)